<a href="https://colab.research.google.com/github/YasmineSarraj/MLOPS-assignments/blob/main/Assignment%203/MLFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pyngrok -q


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [2]:
! pip install mlflow

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [3]:
import mlflow

In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from sklearn.metrics import accuracy_score, mean_squared_error
from statistics import mean

In [5]:
penguins = pd.read_csv("https://github.com/allisonhorst/palmerpenguins/raw/5b5891f01b52ae26ad8cb9755ec93672f49328a8/data/penguins_size.csv")
penguins.dropna(inplace=True)

attributes = penguins.iloc[:,2:6]
scaler = StandardScaler()
num = scaler.fit_transform(attributes)
data_n = pd.DataFrame(num, columns=['culmen_length_mm','culmen_depth_mm','flipper_length_mm','body_mass_g'])

species = penguins.iloc[:, 0]
labelencoder = LabelEncoder()
cat = labelencoder.fit_transform(species)
enc_species = pd.DataFrame(cat, columns=['species_short'])

enc_species = pd.get_dummies(species)


#Tensor Of all the numerical values, lenghth etc. 
X_tensor = torch.tensor(data_n.astype(np.float32).values)

#Tensor of the encoded categorical variable
y_tensor = torch.tensor(enc_species.astype(np.float32).values)



In [6]:
mlflow.set_tracking_uri("sqlite://///Users/yasminesarraj/Documents/GitHub/MLOPS-assignments/Assignment_3/penguins.db")
mlflow.set_experiment("Penguins ANN")
experiment_id = mlflow.get_experiment_by_name('Penguins ANN').experiment_id

Traceback (most recent call last):
  File "/Users/yasminesarraj/Library/Python/3.9/lib/python/site-packages/mlflow/store/tracking/file_store.py", line 290, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/Users/yasminesarraj/Library/Python/3.9/lib/python/site-packages/mlflow/store/tracking/file_store.py", line 383, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/Users/yasminesarraj/Library/Python/3.9/lib/python/site-packages/mlflow/store/tracking/file_store.py", line 1096, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/Users/yasminesarraj/Library/Python/3.9/lib/python/site-packages/mlflow/store/tracking/file_store.py", line 1089, in _read_helper
    result = read_yaml(root, file_name)
  File "/Users/yasminesarraj/Library/Python/3.9/lib/python/site-packages/mlflow/utils/file_utils.py", line 215, in read_yaml
    raise MissingConfigException("Yaml file '

In [7]:
with mlflow.start_run(run_name="run with artifact", nested=True, experiment_id=experiment_id):
  # Your machine learning code here
  # Initializing Hyperparameters
  epochs = 50
  learning_rate = 0.001
  loss_set = {}
  # 1. Creating a FeedForwardNetwork
  model_net4 = torch.nn.Sequential(
                            # Input layer
                            torch.nn.Linear(4,8),
                            torch.nn.ReLU(),
                            # 1. Hidden layer
                            torch.nn.Linear(8,16),
                            torch.nn.ReLU(),
                            # 2. Hidden layer
                            torch.nn.Linear(16,32),
                            torch.nn.ReLU(),
                            # 3. Hidden layer
                            torch.nn.Linear(32,64),
                            torch.nn.ReLU(),
                            # Output layer
                            torch.nn.Linear(64,3),
                            torch.nn.Softmax()  # We have a multiclass single-label classification problem, since a penguin can only be from one species
                          );
  loss_cel = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.RMSprop(model_net4.parameters(), lr=learning_rate)

  # Loop over the number of epochs
  for i in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):
    lost_set_epoch = {}
    j = 0
    for x, y_t in zip(X_tensor, y_tensor):
        output = model_net4.forward(x.float())
        # 2. FeedForward Evaluation
        loss = loss_cel(output, y_t.float())
        optimizer.zero_grad();

        # 3. Backward / Gradient Calculation
        loss.backward()
        # 4. Back Propagation
        optimizer.step()
        # Store the loss for each sample of data
        lost_set_epoch[j] = loss
        j = j + 1
    
    # Store the loss for each epoch
    loss_set[i] = torch.mean(torch.stack(list(lost_set_epoch.values()))).detach().numpy()

    # Display the loss after every 10 epochs
    if (i % 10)==0:
      print (f"Loss: {loss_set[i]}")

  mlflow.log_param("Learning rate", learning_rate)
  mlflow.log_param("Num. of epochs", epochs)

  from pandas.core.groupby.groupby import OutputFrameOrSeries
  with torch.no_grad():
    n_correct = 0
    n_samples = len(y_tensor)

    outputs = model_net4.forward(X_tensor)
    for idx, output in enumerate(outputs):
      pred = np.argmax(output).item()
      true_val = np.argmax(y_tensor[idx]).item()
      
      if pred == true_val:
        n_correct += 1

    acc = 100 * (n_correct / n_samples)

  mlflow.log_metric("accuracy", acc)
  params_path = "model.pt"
  torch.save(model_net4.state_dict(), params_path)
  mlflow.log_artifact(params_path)
  
  # End the current run
  mlflow.end_run()

/var/folders/ss/lhx9lhjx3wn1dkd8thzyx0w40000gn/T/ipykernel_21041/814448772.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):


  0%|          | 0/50 [00:00<?, ?epoch/s]

/Users/yasminesarraj/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Loss: 0.7264402508735657
Loss: 0.5582209229469299
Loss: 0.5569149255752563
Loss: 0.5590516328811646
Loss: 0.5548985600471497


In [8]:
# run tracking UI in the background
get_ipython().system_raw("mlflow ui --port 8080 &")
## Step 3 - Installing pyngrok for remote tunnel access using ngrock.com
!pip install pyngrok --quiet
from pyngrok import ngrok
from getpass import getpass
# Terminate open tunnels if any exist
ngrok.kill()
## Step 4 - Login on ngrok.com and get your authtoken from https://dashboard.ngrok.com/auth
# Enter your auth token when the code is running
NGROK_AUTH_TOKEN = getpass('Enter the ngrok authtoken: ')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok_tunnel = ngrok.connect(addr="8080", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

[2023-05-02 16:31:49 +0200] [21095] [INFO] Starting gunicorn 20.1.0
[2023-05-02 16:31:49 +0200] [21095] [ERROR] Connection in use: ('127.0.0.1', 8080)
[2023-05-02 16:31:49 +0200] [21095] [ERROR] Retrying in 1 second.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


[2023-05-02 16:31:50 +0200] [21095] [ERROR] Connection in use: ('127.0.0.1', 8080)
[2023-05-02 16:31:50 +0200] [21095] [ERROR] Retrying in 1 second.
[2023-05-02 16:31:51 +0200] [21095] [ERROR] Connection in use: ('127.0.0.1', 8080)
[2023-05-02 16:31:51 +0200] [21095] [ERROR] Retrying in 1 second.
[2023-05-02 16:31:52 +0200] [21095] [ERROR] Connection in use: ('127.0.0.1', 8080)
[2023-05-02 16:31:52 +0200] [21095] [ERROR] Retrying in 1 second.
[2023-05-02 16:31:53 +0200] [21095] [ERROR] Connection in use: ('127.0.0.1', 8080)
[2023-05-02 16:31:53 +0200] [21095] [ERROR] Retrying in 1 second.
[2023-05-02 16:31:54 +0200] [21095] [ERROR] Can't connect to ('127.0.0.1', 8080)
Running the mlflow server failed. Please see the logs above for details.


MLflow Tracking UI: https://ad16-86-52-72-48.ngrok-free.app
